# Problem Definition & Objective
Selected Project Track: AI in Personalized Learning

I have built an algorithm which allows the user (medical student) to attempt questions related to neurology, provides explanations for incorrect answers, uses reinforcement learning to present relevant questions to the user, and eventually provides an analysis of which topics the student needs to work on.


Neurology is a branch of medicine that deals with diseases of the human nervous system, including the brain, spinal cord and nerves. Medical students often find it hard to learn and memorise concepts related to this field, because of the complexity of concepts and confusing terminology. In my personal experience, a Q&A based learning tool is the ideal resource to help students retain information better.



# Data Understanding & Preparation
Data used for developing this project was synthetically generated using ChatGPT 5.2. The chatbot was initially prompted to generate a dataset of 50 multiple-choice questions from the field of neurology (more specifically, neuroanatomy and its clinical aspects). These questions were accompanied by answers and short explanations about the answers. ChatGPT was told to ensure that the difficulty level of questions was equally distributed between easy, moderate and hard. The data was then expanded to 200 questions, with the condition that the content must include all aspects of neurology in comparable quantity. The final dataset used was in the form of a .csv file, which included columns for question; options a, b, c and d; correct option; topic; difficulty; and explanation.

##Data Loading

In [32]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
quiz_data = pd.read_csv('/content/ProjectDataSet.csv')


##Data Exploration

Since this data was specifically generated for this project, it is a clean dataset not needing any traditional data cleaning or handling of noise.

In [33]:
available_questions = quiz_data.to_dict(orient='records')

topic_performance = {}
for topic in quiz_data['topic'].unique():
    topic_performance[topic] = {'correct_count': 0, 'incorrect_count': 0, 'attempted': 0}

print(f"Initialized {len(available_questions)} questions in available_questions.")
print(f"Initialized {len(topic_performance)} topics in topic_performance with counts set to zero.")

Initialized 200 questions in available_questions.
Initialized 39 topics in topic_performance with counts set to zero.


# Model Design

Heuristic Optimization, Greedy Policies and Stochastic Decision Processes

In [34]:
import random

def get_adaptive_question(available_questions, topic_performance):
    if not available_questions:
        return None

    # Identify weak topics (those with the highest incorrect_count)
    weak_topics = []
    max_error_rate = -1

    for topic, perf in topic_performance.items():
        if perf['attempted'] == 0:
            continue   # PREVENTS division by zero

        error_rate = perf['incorrect_count'] / perf['attempted']

        if error_rate > max_error_rate:
            max_error_rate = error_rate
            weak_topics = [topic]
        elif error_rate == max_error_rate:
            weak_topics.append(topic)

    '''
    If all topics have 0 incorrect answers or no incorrect answers recorded yet,
    or if max_incorrect_count is still -1 (meaning topic_performance is empty or all counts are 0),
    we will treat all topics equally or pick randomly from all available.
    '''

    # Try to find a question from weak topics first
    if weak_topics: # Only prioritize if there are actual incorrect answers
        prioritized_questions = [q for q in available_questions if q['topic'] in weak_topics]
        if prioritized_questions:
            selected_question = random.choice(prioritized_questions)
            available_questions.remove(selected_question)
            return selected_question

    '''
    Fallback: If no weak topics (or no questions in them) or all topics are equally 'strong' (0 incorrect answers),
    pick a random question from the remaining available questions.
    '''
    selected_question = random.choice(available_questions)
    available_questions.remove(selected_question)
    return selected_question

##Justification of design choices
1. Capping each session at 50 questions (terminating loops accordingly) - to ensure diversity of knowledge and prevent cognitive fatigue
2. Tracking performance for each sub-topic within Neurology - to help target weaker areas rather than repetitively learing everything
3. Preferentially selecting questions from weaker topics i.e. those with high error rates - same as above
4. Representing topic performance as a ratio (correct / attempted) - to normalize performance assessment across unequally distributed data
5. Hybrid logic with deterministic and probabilistic processes - keeps behaviour easy to interpret while not being predictable

# Core Implementation

In [35]:
score = 0
correct_answers_count = 0
incorrect_answers_count = 0
question_results = [] # To store detailed results for analysis

question_number = 0
max_questions = 25
while available_questions and question_number < max_questions:
    question_number += 1
    current_question = get_adaptive_question(available_questions, topic_performance)

    if current_question is None:
        print("No more questions available.")
        break

    print(f"\nQuestion {question_number}: {current_question['question_text']}")
    print(f"A: {current_question['option_a']}")
    print(f"B: {current_question['option_b']}")
    print(f"C: {current_question['option_c']}")
    print(f"D: {current_question['option_d']}")

    while True:
        user_answer = input("Your answer (A, B, C, or D): ").strip().lower()
        if user_answer in ['a', 'b', 'c', 'd']:
            break
        else:
            print("Invalid input. Please enter A, B, C, or D.")

    correct_answer_char = current_question['correct_option'].lower()
    is_correct = (user_answer == correct_answer_char)

    # Update topic_performance
    question_topic = current_question['topic']
    topic_performance[question_topic]['attempted'] += 1
    if is_correct:
        print("Correct!")
        score += 1
        correct_answers_count += 1
        topic_performance[question_topic]['correct_count'] += 1
    else:
        print(f"Incorrect. The correct answer was {correct_answer_char.upper()}.")
        incorrect_answers_count += 1
        topic_performance[question_topic]['incorrect_count'] += 1
        if 'explanation' in current_question:
            print(f"Explanation: {current_question['explanation']}")
        else:
            print("Explanation unavailable.")

    question_results.append({
        'id': current_question['id'],
        'topic': question_topic,
        'is_correct': is_correct
    })

    # Real-time summary after each question
    print("\n--- Current Progress ---")
    print(f"Questions Answered: {question_number}")
    print(f"Current Score: {score}")
    print(f"Correct Answers: {correct_answers_count}")
    print(f"Incorrect Answers: {incorrect_answers_count}")
    current_accuracy = (correct_answers_count / question_number) * 100 if question_number > 0 else 0
    print(f"Current Accuracy: {current_accuracy:.2f}%")

final_total_questions = correct_answers_count + incorrect_answers_count
print(f"\nQuiz finished! Your final score is: {score}/{final_total_questions}")



Question 1: The posterior limb of the internal capsule primarily carries:
A: Visual signals
B: Motor fibers
C: Cerebellar signals
D: Pain fibers


KeyboardInterrupt: Interrupted by user

In [ ]:
print("\n--- Adaptive Quiz Performance Summary ---")

# Overall Quiz Performance
final_total_questions = correct_answers_count + incorrect_answers_count
if final_total_questions > 0:
    overall_accuracy = (correct_answers_count / final_total_questions) * 100
else:
    overall_accuracy = 0

print(f"Total Questions Attempted: {final_total_questions}")
print(f"Overall Correct Answers: {correct_answers_count}")
print(f"Overall Incorrect Answers: {incorrect_answers_count}")
print(f"Overall Accuracy: {overall_accuracy:.2f}%\n")

# Detailed Topic-wise Performance
print("--- Topic-wise Performance Analysis ---")

topic_accuracies = {}
for topic, performance in topic_performance.items():
    topic_total = performance['correct_count'] + performance['incorrect_count']
    if topic_total > 0:
        topic_accuracy = (performance['correct_count'] / topic_total) * 100
        topic_accuracies[topic] = {
            'total_attempted': topic_total,
            'correct': performance['correct_count'],
            'incorrect': performance['incorrect_count'],
            'accuracy': topic_accuracy
        }
        print(f"Topic: {topic}")
        print(f"  Questions Attempted: {topic_total}")
        print(f"  Correct: {performance['correct_count']}")
        print(f"  Incorrect: {performance['incorrect_count']}")
        print(f"  Accuracy: {topic_accuracy:.2f}%\n")
    else:
        # Only print topics that had questions attempted during the quiz
        if performance['correct_count'] > 0 or performance['incorrect_count'] > 0:
            print(f"Topic: {topic} (No questions attempted or issues in recording performance)\n")

# Identify Weakest Topics
if topic_accuracies:
    sorted_topics = sorted(topic_accuracies.items(), key=lambda item: item[1]['accuracy'])

    print("--- Weakest Topics (by accuracy) ---")
    # Print up to 5 weakest topics, or fewer if not many topics were attempted
    for i, (topic, data) in enumerate(sorted_topics):
        if i >= 5: # Limit to top 5 weakest topics
            break
        if data['total_attempted'] > 0 and data['accuracy'] < 100:
            print(f"- {topic}: {data['incorrect']} incorrect out of {data['total_attempted']} (Accuracy: {data['accuracy']:.2f}%)")

    if not any(data['accuracy'] < 100 and data['total_attempted'] > 0 for data in topic_accuracies.values()):
        print("Great job! No incorrect answers in any topic.")
else:
    print("No topic-wise data available for analysis.")



# Evaluation and Analysis
Performance analysis

The performance analysis of this adaptive learning model focuses on how effectively it adapts question difficulty and topic selection and how much it improves learning outcomes in neurology. Since the model provides both real-time feedback and comprehensive post-session analysis, performance is assessed across adaptivity, learning gain, and diagnostic capability.

Parameters of evaluation
1. Efficient question selection: Reflecting the model's ability to personalize, minimise redundancy while maximizing educational value.
2. Significant learning gains: Measured as the model's ability to provide adaptive sequencing and feedback mechanisms inorder to actively contribute to improved understanding in place of traditional passive assessment.

As this is a basic prototype, currently this is an intutive analysis performed by the developer in the test runs.

# Limitations and Ethical Considerations
This tool can be significantly refined to perform optimally. The dataset that I have used to build it currently, has been synthetically generated by ChatGPT and is also a small dataset of 200 questions with answers and explanations. The scope of this project is currently limited only to Neurology but can be expanded to include other specialties of medicine. There are no concerns about the ethical implications of the tool as it does not require any personal data from the user, and the model design does not inherently predispose to bias.

# Conclusion and Future Scope
In conclusion, I created a tool to help medical students learn about Neurology and understand what topics they are weak in. Through this process, I improved my understanding of coding and Artificial Intelligence.

Future improvements to the tool could include introducing a spaced repetition system for questions answered incorrectly, or provision of learning resources or explanations for topics that appear to be persistently weak, or allowing users to focus on specific weak topics identified in the performance summary.
A better user interface (UI) can also be developed, along with a gamification layer, to make the experience more enjoyable for the user.